In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
sns.set_palette("tab20")

from scipy import stats

from covis.utils import get_project_root

# data wrangling to extract total number of deaths England & Wales 2017 - 2023

## 2023

In [ ]:
weekly_2023 = pd.read_excel(
    get_project_root() / "data/publicationfileweek172023.xlsx",
    sheet_name=3,
    header=5
)

In [ ]:
print(weekly_2023.shape)
weekly_2023.head()

In [ ]:
weekly_2023.dtypes

In [ ]:
weekly_2023.groupby("Week ended")["Total deaths England and Wales (2023)"].sum()

In [ ]:
weekly_2023 = weekly_2023[[
    "Week number",
    "Week ended",
    "Total deaths England and Wales (2023)",
]].set_index("Week number")

weekly_2023.rename(columns={"Total deaths England and Wales (2023)":"Total deaths, all ages (2023)"}, inplace=True)

In [ ]:
print(weekly_2023.shape)
weekly_2023.head()

## 2022

In [ ]:
weekly_2022 = pd.read_excel(
    get_project_root() / "data/publicationfileweek522022.xlsx",
    sheet_name=3,
    header=5
)

In [ ]:
print(weekly_2022.shape)
weekly_2022.head()

In [ ]:
weekly_2022.dtypes

In [ ]:
weekly_2022 = weekly_2022[["Week number", "Week ended", "Total deaths England and Wales (2022)"]].set_index("Week number")
weekly_2022.rename(columns={"Total deaths England and Wales (2022)":"Total deaths, all ages (2022)"}, inplace=True)

In [ ]:
print(weekly_2022.shape)
weekly_2022.head()

# 2021
* sheet has a different format, so alter the read in commands

In [ ]:
# find the rows to read in by trail:
weekly_2021 = pd.read_excel(
    get_project_root() / "data/publishedweek522021.xlsx",
    sheet_name="Weekly figures 2021",
    skiprows=lambda x: x not in [4,5,8],
    usecols=lambda x: x not in ["53 7", "Unnamed: 1"]
)

In [ ]:
weekly_2021

In [ ]:
weekly_2021 = weekly_2021.set_index("Week number").transpose()

In [ ]:
print(weekly_2021.shape)
weekly_2021.head()

In [ ]:
weekly_2021.dtypes

## 2020

In [ ]:
weekly_2020 = pd.read_excel(
    get_project_root() / "data/publishedweek532020.xlsx",
    sheet_name="Weekly figures 2020",
    skiprows=lambda x: x not in [4,5,8],
    usecols=lambda x: x not in ["Unnamed: 1"]
)

weekly_2020 = weekly_2020.set_index("Week number").transpose()
weekly_2020.rename(columns={"Total deaths, all ages":"Total deaths, all ages (2020)"}, inplace=True)

print(weekly_2020.shape)
weekly_2020.tail()

## 2019

In [ ]:
weekly_2019 = pd.read_excel(
    get_project_root() / "data/publishedweek522019.xls",
    sheet_name="Weekly figures 2019",
    # skiprows=lambda x: x not in range(10),
    skiprows=lambda x: x not in [3,4,7],
    usecols=lambda x: x not in ["Unnamed: 1"]
)

weekly_2019 = weekly_2019.set_index("Week number").transpose()
weekly_2019.rename(columns={"Total deaths, all ages":"Total deaths, all ages (2019)"}, inplace=True)

print(weekly_2019.shape)
weekly_2019.head()

## 2018

In [ ]:
weekly_2018 = pd.read_excel(
    get_project_root() / "data/publishedweek522018withupdatedrespiratoryrow.xls",
    sheet_name="Weekly figures 2018",
    # skiprows=lambda x: x not in range(10),
    skiprows=lambda x: x not in [3,4,7],
    usecols=lambda x: x not in ["Unnamed: 1"]
)

weekly_2018 = weekly_2018.set_index("Week number").transpose()
weekly_2018.rename(columns={"Total deaths, all ages":"Total deaths, all ages (2018)"}, inplace=True)

print(weekly_2018.shape)
weekly_2018.head()

# 2017

In [ ]:
weekly_2017 = pd.read_excel(
    get_project_root() / "data/publishedweek522017.xls",
    sheet_name="Weekly figures 2017",
    # skiprows=lambda x: x not in range(10),
    skiprows=lambda x: x not in [3,4,7],
    usecols=lambda x: x not in ["Unnamed: 1"]
)

weekly_2017 = weekly_2017.set_index("Week number").transpose()
weekly_2017.rename(columns={"Total deaths, all ages":"Total deaths, all ages (2017)"}, inplace=True)

print(weekly_2017.shape)
weekly_2017.head()

In [ ]:
df = weekly_2023.rename(columns={c:c[:-7] for c in weekly_2023.columns if c.startswith("Total deaths")})
for year_weekly in [weekly_2022, weekly_2021, weekly_2020, weekly_2019, weekly_2018, weekly_2017]:
    df = pd.concat(
        [df, year_weekly.rename(columns={c:c[:-7] for c in year_weekly.columns if c.startswith("Total deaths")})]
    )

# df.to_csv(
#     get_project_root() / "output/total_deaths_per_week_2017-2023.csv"
# )

print(df.shape)
df.head()

In [ ]:
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(111)

sns.lineplot(
    data=df,
    x="Week ended",
    y="Total deaths, all ages",
    estimator="mean",
    ax=ax,
    label="weekly total deaths"
)
ax.axhline(
    y=df["Total deaths, all ages"].mean(),
    c="tab:orange",
    ls="--",
    label="average"
)

ax.set_ylabel("Total Deaths")
ax.set_xlabel("")
ax.legend()

fig.suptitle("Total Weekly Deaths, England & Wales (2017-2023)")
fig.text(0.1,-0.05,"$^{*}$average is calculated over the plotted period")
fig.tight_layout();

In [ ]:
# fig.savefig(
#     get_project_root() / "figures/total_weekly_deaths.png"
# )

## hypothesis test for increase in average deaths per month

In [ ]:
df.dtypes

In [ ]:
df["Total deaths, all ages"] = pd.to_numeric(df["Total deaths, all ages"])

In [ ]:
monthly_deaths = df.set_index("Week ended").groupby(pd.Grouper(freq="M")).sum()

In [ ]:
monthly_deaths.loc["01/01/2017":"12/31/2019"]["Total deaths, all ages"].plot(kind="hist");

In [ ]:
monthly_deaths.loc["01/01/2020":"12/31/2020"]["Total deaths, all ages"].plot(kind="hist");

* will probably have to look at weekly numbers to assume some normality in the distribution

In [ ]:
df.set_index("Week ended").loc["01/01/2017":"12/31/2019"]["Total deaths, all ages"].plot(kind="hist");

In [ ]:
df.set_index("Week ended").loc["01/01/2020":"12/31/2020"]["Total deaths, all ages"].plot(kind="hist");

* still not fantastic

In [ ]:
g1_weekly_2017_2020 = df.set_index("Week ended").loc["01/01/2017":"12/31/2019"]["Total deaths, all ages"]

In [ ]:
# g1_weekly_2017_2020

In [ ]:
g2_weekly_2020_2021 = df.set_index("Week ended").loc["01/01/2020":"12/31/2021"]["Total deaths, all ages"]

In [ ]:
# g2_weekly_2020_2021

In [ ]:
g1_weekly_2017_2020.mean()

In [ ]:
g2_weekly_2020_2021.mean()

In [ ]:
t_test = stats.ttest_ind(g1_weekly_2017_2020, g2_weekly_2020_2021)
t_test

* for one tailed t-test divide p-value by 2

In [ ]:
t_stat, p_val = t_test

In [ ]:
one_tailed_p = p_val / 2

In [ ]:
one_tailed_p